# 面向对象的高级编程

数据封装、继承和多态只是面向对象程序设计中最基础的3个概念。在Python中，面向对象还有很多高级特性，允许我们写出非常强大的功能。

我们会讨论多重继承、定制类、元类等概念。


## 使用`__slots__`来限制对类的属性扩展、

我们知道由于Python的动态语言特性，我们可以很容易很一个类的实例扩展新的属性，同时我们也可以扩展新的方法。

In [1]:
class Student(object):
    pass
def set_age(self, age):
    self.age = age
s = Student()
from types import MethodType
s.set_age = MethodType(set_age, s)
s.set_age(25)
s.age

25

但是对实例绑定的方法，对别一个实例是不起作用的：

In [2]:
s2 = Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有的实例都绑定方法，可以给class绑定方法：

In [4]:
def set_score(self, score):
    self.score = score
# 为所有的实例都绑定set_score方法
Student.set_score = set_score
# 为所有实例都绑定age属性
Student.age = 23
print(s2.age)
s2.set_score(32)
print(s2.score)

23
32


通常情况下，我们一般把上面的`set_score`和`age`写到类的定义里，但动态绑定允许我们在程序运行的过程中动态给class加上功能，这在静态语言中是很难实现的。

那如果我们想限制用户对实例的属性扩展，怎么办呢？Python允许在定义class的时候，定义一个特殊的`__slots__`变量，来限制该class实例能添加的属性：

In [5]:
class Student(object):
    __slots__ = ('name', 'age')

s = Student()
s.name = 'Michael'
s.age = 25
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

由于`score`没有被放到`__slots__`中，所以不能绑定`score`属性

使用`__slots__`要注意的是，`__slots__`定义的属性仅对当前类实例起作用，对继承的子类是不起作用的。

In [6]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 999

除非在子类中也定义`__slots__`，这样，子类实例允许定义的属性就是**自身的`__slots__`加上父类的`__slots__`**

In [7]:
class GraduateStudent(Student):
    __slots__ = ('score')
g = GraduateStudent()
g.name = 'Michael'
g.age = 25
g.score = 99

## 使用`@property`

我们可以通过对象直接set/get属性值，这样写起来很简单，但是没办法检查参数

In [8]:
class Student(object):
    def level(self):
        if self.score > 90:
            return 'A'
        elif self.score > 70:
            return 'B'
        elif self.score > 60:
            return 'C'
        else:
            return 'D'
s = Student()
s.score = 'abc'
print(s.level())

TypeError: '>' not supported between instances of 'str' and 'int'

上面的代码中，我们本来期望score是一个int型，但是如果使用时给score赋予了str，并不会报错。埋下了一个潜在的风险。

我们可以通过约束，通过接口来对属性进行set和get

In [9]:
class Student(object):
    def set_score(self, score):
        if not isinstance(score, int):
            raise ValueError('score must be an integer')
        if score < 0 or score > 100:
            raise ValueError('score must between 0~100!')
        self.__score = score
    def get_score(self):
        return self.__score
s = Student()
s.set_score(60)
print(s.get_score())
s.set_score('abc')

60


ValueError: score must be an integer

使用上面的方法，虽然解决了问题，但是调用又略显复杂，没有直接用属性那么简单直接。下面我们就可以使用装饰器`property`来实现这个目的。

把一个`getter`方法变成属性，只需要加上`@property`就可以了，此时，`@property`本身又创建了另一个装饰器`@score.setter`，负责把一个`setter`方法变成属性赋值

In [10]:
class Student(object):
    @property
    def score(self):
        return self.__score
    @score.setter
    def score(self, score):
        if not isinstance(score, int):
            raise ValueError('score must be an integer')
        if score < 0 or score > 100:
            raise ValueError('score must between 0~100!')
        self.__score = score
s = Student()
s.score = 32
print(s.score)
s.score = 101

32


ValueError: score must between 0~100!

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性

下面的代码示例中，`level`就是一个只读的属性。

In [11]:
class Student(object):
    @property
    def score(self):
        return self.__score
    @score.setter
    def score(self, score):
        if not isinstance(score, int):
            raise ValueError('score must be an integer')
        if score < 0 or score > 100:
            raise ValueError('score must between 0~100!')
        self.__score = score
    @property
    def level(self):
        if self.score > 90:
            return 'A'
        elif self.score > 70:
            return 'B'
        elif self.score > 60:
            return 'C'
        else:
            return 'D'
s = Student()
s.score = 99
print(s.level)
s.level = 'D'

A


AttributeError: can't set attribute

## 多重继承

某些情况下，我们需要某一种对象同时具体多种特征，这里我们可以采用多重继承来实现。

In [12]:
#动物
class Animal(object):
    pass
# 能地上跑的
class Runnable(Animal):
    pass
# 能天上飞的
class Flyable(Animal):
    pass
# 哺乳动物
class Mammal(Animal):
    pass
# 鸟类
class Bird(Animal):
    pass

# 蝙蝠属于哺乳动物，同时属于能飞的
class Bat(Mammal, Flyable):
    pass

如果不使用多重继承，只使用单一层次，那就要定义：能跑的哺乳动

## 一些特殊用处的函数

- `__len__()`可以该class的对象作用于`len()`函数
- `__str__()`可以自定义该class的对象print出来的结果
- `__repr__()`可以自定义该class的对象调试打印的结果
- `__iter__()`如果一个类想被用于`for...in`循环，类似list或tuple那样，就必须实现该方法，该方法返回一个迭代对象。然后，Python的for循环就会不断调用该迭代对象的
- `__next__()`方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。
- `__getitem__`可以让该class的对象用`[]`下标来访问。
- `__setitem__()`
- `__delitem__()`
- `__getattr__()`
- `__call()`

上面只列了一些常见的特殊函数，Python只持非常多这种特殊函数，为了是让用户自定义的类型，可以表现的内置类型一样。

具体可以参考Python的官方文档：[Special method names](http://docs.python.org/3/reference/datamodel.html#special-method-names)

## 枚举类

## 元类